In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import sys
sys.path.insert(1, 'C:/Users/pnknv/Documents/GitHub/yadisk_loader')
from main_utils import *

In [3]:
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config()

In [4]:
sql_server, sql_engine = get_sql_engine()

In [5]:
city_df, source_df, exc_code = get_tables_info(sql_engine)

In [6]:
city_df

,id,name,url_avito,population
0,1,Владивосток,vladivostok,0
1,2,Краснодар,krasnodar,0
2,3,Сочи,sochi,0
3,4,Новороссийск,novorossiysk,0
4,5,Анапа,anapa,0
5,6,Армавир,armavir,0
6,7,Челябинск,chelyabinsk,0
7,8,Магнитогорск,magnitogorsk,0
8,9,Миасс,miass,0
9,10,Копейск,kopeysk,0


# list files

In [7]:
files = os.listdir('saved_shit/')

In [8]:
files_avito = [i for i in files if 'циан' not in i]
files_cian = [i for i in files if 'циан' in i]

In [9]:
test_cian = pd.read_csv('saved_shit/' + files_cian[0], delimiter=',')

In [16]:
test_cian['city'] = test_cian['Адрес'].apply(lambda x: x.split(';')[1])

In [45]:
test_avito = pd.read_csv('saved_shit/' + files_avito[0], delimiter=';')

In [46]:
test_avito['city'] = test_avito['Ссылка'].apply(lambda x: x.split('/')[3])

In [41]:
def count_unique_addr(df, cities):
    cities = cities[1:]
    val = []
    for city in cities:
        val.append(len(df[df.city == city]['Адрес'].unique()))
    return val

# avito

In [42]:
city_list = test_avito.city.unique().tolist()
city_list.insert(0, 'date')
df_avito = pd.DataFrame(columns=city_list)
for file in files_avito:
    test_avito = pd.read_csv('saved_shit/' + file, delimiter=';')
    test_avito['Адрес'] = test_avito.apply(lambda row: avito_addr_from_row(row, city_df), axis=1)
    test_avito['city'] = test_avito['Ссылка'].apply(lambda x: x.split('/')[3])
    date = file[:8]
    counts = count_unique_addr(test_avito, city_list)
    counts.insert(0, date)
    df_avito.loc[len(df_avito)] = counts 

In [43]:
df_avito

,date,tomsk,armavir,chelyabinsk,novorossiysk,kazan,krasnodar,novosibirsk,krasnoyarsk,sochi
0,08-09-23,1297,317,2477,1046,3694,6659,5129,2681,5626
1,09-09-23,1190,314,2560,1022,3650,6745,5050,2812,5681
2,10-09-23,1260,308,2511,990,3646,6494,4997,2689,5722
3,11-09-23,1329,323,2586,1055,3627,6682,5150,2815,5728
4,12-09-23,1327,315,2493,1042,3650,6896,5172,2860,5731
5,13-09-23,1262,305,2442,1026,3659,6719,5226,2690,5606
6,14-09-23,1327,330,2535,1017,3707,6518,5121,2594,5454
7,15-09-23,1264,327,2478,982,3682,6702,5027,2679,4978
8,16-09-23,1261,313,2501,997,3620,6455,5098,2701,5724
9,17-09-23,1217,333,2512,1013,3586,6548,5067,2678,5746


In [55]:
df_avito.to_excel('avito_stats.xlsx')

# cian

In [53]:
city_list = test_cian.city.unique().tolist()
city_list = [i for i in city_list if i in city_df.name.to_list()]
city_list.insert(0, 'date')
df_cian = pd.DataFrame(columns=city_list)
for file in files_cian:
    test_cian = pd.read_csv('saved_shit/' + file, delimiter=',')
    test_cian['city'] = test_cian['Адрес'].apply(lambda x: x.split(';')[1])
    date = file[5:13]
    counts = count_unique_addr(test_cian, city_list)
    counts.insert(0, date)
    df_cian.loc[len(df_cian)] = counts 

In [54]:
df_cian

,date,Сочи,Новороссийск,Краснодар,Новосибирск,Казань,Саратов,Калининград,Челябинск,Томск,Анапа,Армавир,Копейск,Красноярск
0,08-09-23,3628,894,4287,7887,3697,1663,2013,1978,1526,932,119,0,2152
1,09-09-23,3652,893,4269,7990,3749,1665,2013,1944,1530,957,118,0,2136
2,10-09-23,3414,891,4263,7946,3737,1640,2012,1949,1534,962,118,0,2132
3,11-09-23,3600,873,4252,7489,3702,1640,2005,1944,1508,957,120,2,2154
4,12-09-23,3551,903,4256,7860,3762,1662,2024,1927,1510,955,119,0,2159
5,13-09-23,3610,898,4260,7846,3733,1631,2024,1919,1484,939,119,0,2179
6,14-09-23,3387,904,4279,7880,3066,1643,2021,1891,1502,948,116,0,2163
7,15-09-23,3470,907,4269,7825,3695,1522,2025,1848,1470,942,116,0,2159
8,16-09-23,3613,909,4274,7900,3698,1646,2028,1870,1510,962,123,0,2175
9,17-09-23,3657,907,4284,7820,3670,1649,2024,1853,1505,961,118,0,2156


In [56]:
df_cian.to_excel('cian_stats.xlsx')

In [7]:
cian['ad_id'] = cian['Ссылка'].apply(lambda x: x.split('/')[-2])

In [8]:
cian['ad_id'] = cian['ad_id'].astype('int64')

In [15]:
def get_exist_ad_id(engine, source):
    source_id = 2 if source == 'cian' else 3
    ad_id_query = f'SELECT ad_id FROM realty where source_id = {source_id}'
    try:
        con_obj = engine.connect()
        ad_id_db = pd.read_sql(text(ad_id_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        logging.error(traceback.format_exc())
        ad_id_db = None
        exc_code = exc.code
    return ad_id_db, exc_code

In [16]:
ads_prod, exc_code = get_exist_ad_id(sql_engine, 'cian')

In [17]:
len(cian[~cian.ad_id.isin(ads_prod.ad_id.to_list())])

20

# money

In [57]:
files = os.listdir('saved_shit/')

In [58]:
files_avito = [i for i in files if 'циан' not in i]
files_cian = [i for i in files if 'циан' in i]

In [59]:
test_cian = pd.read_csv('saved_shit/' + files_cian[0], delimiter=',')

In [60]:
test_cian['ad_id'] = test_cian['Ссылка'].apply(lambda x: x.split('/')[-2])
test_cian['ad_id'] = test_cian['ad_id'].astype('int64')

In [61]:
test_cian

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Адрес,Дата публикации,Тип продавца,Продавец,ad_id
0,https://krasnodar.cian.ru/sale/flat/286046952/,"2-комн. кв., 70 м², 5/12 этаж",2-комн. кв.,70 м²,5/12 этаж,5700000,81429,ЖК «Парус»,Краснодарский край;Краснодар;Прикубанский;мкр....,"вчера (вчера, 12:55)",Агентство недвижимости,ЧФК-Недвижимость,286046952
1,https://krasnodar.cian.ru/sale/flat/286032732/,"2-комн. кв., 51,5 м², 2/10 этаж",2-комн. кв.,"51,5 м²",2/10 этаж,5700000,110680,NaN,Краснодарский край;Краснодар;Прикубанский;мкр....,"пять дней назад (25 апр, 18:22)",Агентство недвижимости,ЭТАЖИ-КУБАНЬ,286032732
2,https://krasnodar.cian.ru/sale/flat/286032730/,"1-комн. кв., 40 м², 7/22 этаж",1-комн. кв.,40 м²,7/22 этаж,5700000,142500,NaN,Краснодарский край;Краснодар;Прикубанский;улиц...,"пять дней назад (25 апр, 18:22)",Агентство недвижимости,ЭТАЖИ-КУБАНЬ,286032730
3,https://krasnodar.cian.ru/sale/flat/286029930/,"1-комн. кв., 47,8 м², 14/20 этаж",1-комн. кв.,"47,8 м²",14/20 этаж,5700000,119247,ЖК «Грани»,Краснодарский край;Краснодар;Прикубанский;улиц...,"2 недели назад (11 апр, 13:58)",Риелтор,ID 55338816,286029930
4,https://krasnodar.cian.ru/sale/flat/286020566/,"3-комн. кв., 66 м², 7/9 этаж",3-комн. кв.,66 м²,7/9 этаж,5700000,86364,ЖК «Sport Village»,Краснодарский край;Краснодар;Прикубанский;мкр....,"неделю назад (20 апр, 13:23)",Агентство недвижимости,Девелопмент-Юг,286020566
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117291,https://sochi.cian.ru/sale/flat/277480315/,"3-комн. кв., 60 м², 2/11 этаж",3-комн. кв.,60 м²,2/11 этаж,13000000,216667,NaN,Краснодарский край;Сочи;р-н Центральный;мкр. Д...,"четыре дня назад (27 апр, 09:18)",Собственник,ID 94977241,277480315
117292,https://sochi.cian.ru/sale/flat/277459856/,"3-комн. кв., 101 м², 11/17 этаж",3-комн. кв.,101 м²,11/17 этаж,12300000,121782,NaN,Краснодарский край;Сочи;р-н Центральный;мкр. Ц...,"2 недели назад (13 апр, 13:54)",Агентство недвижимости,Черноморский проспект,277459856
117293,https://sochi.cian.ru/sale/flat/277438444/,"Апартаменты-студия, 25 м², 4/5 этаж",Апартаменты-студия,25 м²,4/5 этаж,12500000,500000,NaN,Краснодарский край;Сочи;р-н Центральный;мкр. Ц...,"2 недели назад (17 апр, 17:58)",Риелтор,Виктор Захаров,277438444
117294,https://sochi.cian.ru/sale/flat/277347516/,"Студия, 12 м², 4/4 этаж",Студия,12 м²,4/4 этаж,13000000,1083333,NaN,Краснодарский край;Сочи;р-н Центральный;мкр. Н...,"полгода назад (19 сен, 23:58)",Собственник,ID 93157894,277347516


In [62]:
test_avito = pd.read_csv('saved_shit/' + files_avito[0], delimiter=';')

In [63]:
test_avito['ad_id'] = test_avito['Ссылка'].apply(lambda x: x.split('_')[-1])
test_avito['ad_id'] = test_avito['ad_id'].astype('int64')

In [64]:
test_avito

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений,ad_id
0,https://www.avito.ru/tomsk/kvartiry/apartament...,"Апартаменты-студия, 29 м², 5/5 эт.",Апартаменты-студия,29 м²,5/5 эт.,1975000,68103,NaN,р-н Ленинский,"микрорайон Черемошники, Кузнецкий пер., 5",12 часов назад,Частное лицо,NaN,NaN,NaN,3169985711
1,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 34,9 м², 4/6 эт.",1-к. квартира,"34,9 м²",4/6 эт.,3700000,106017,NaN,р-н Кировский,"ул. Матросова, 6",6 дней назад,Частное лицо,NaN,NaN,NaN,2739762184
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 36 м², 1/2 эт.",2-к. квартира,36 м²,1/2 эт.,2600000,72222,NaN,р-н Октябрьский,"ул. Мичурина, 99Ас4",22 часа назад,Агентство,ОФИС ПРОДАЖ «ЗАСТРОЙЩИКА»,https://www.avito.ru/user/c245de34b6855769043d...,388,2861453708
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 37 м², 3/5 эт.",2-к. квартира,37 м²,3/5 эт.,3350000,90541,NaN,р-н Ленинский,"ул. Говорова, 18",4 часа назад,Частное лицо,NaN,NaN,NaN,2859068275
4,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 44,8 м², 3/5 эт.",2-к. квартира,"44,8 м²",3/5 эт.,5400000,120536,NaN,р-н Кировский,"ул. Гоголя, 61",2 часа назад,Частное лицо,NaN,NaN,NaN,2920724735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200832,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 7/24 эт.",2-к. квартира,"64,2 м²",7/24 эт.,11877000.0,185000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2919396604
200833,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 8/24 эт.",2-к. квартира,"64,2 м²",8/24 эт.,11877000.0,185000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2918873627
200834,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 6/24 эт.",2-к. квартира,"64,2 м²",6/24 эт.,11877000.0,185000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2918970728
200835,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 5/24 эт.",2-к. квартира,"64,2 м²",5/24 эт.,11556000.0,180000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2919298481


In [65]:
files_avito = sorted(files_avito, key=lambda x: datetime.strptime(x[:8], '%d-%m-%y'))
files_cian = sorted(files_cian, key=lambda x: datetime.strptime(x[5:13], '%d-%m-%y'))

In [66]:
files_avito

['25-04-23 (без дублей).csv',
 '26-04-23 (без дублей).csv',
 '27-04-23 (без дублей).csv',
 '28-04-23 (без дублей).csv',
 '29-04-23 (без дублей).csv',
 '30-04-23 (без дублей).csv',
 '01-05-23 (без дублей).csv',
 '02-05-23 (без дублей).csv',
 '03-05-23 (без дублей).csv',
 '04-05-23 (без дублей).csv',
 '05-05-23 (без дублей).csv',
 '06-05-23 (без дублей).csv',
 '07-05-23 (без дублей).csv',
 '08-05-23 (без дублей).csv',
 '09-05-23 (без дублей).csv',
 '10-05-23 (без дублей).csv',
 '11-05-23 (без дублей).csv',
 '12-05-23 (без дублей).csv',
 '13-05-23 (без дублей).csv',
 '14-05-23 (без дублей).csv',
 '15-05-23 (без дублей).csv',
 '16-05-23 (без дублей).csv',
 '17-05-23 (без дублей).csv',
 '18-05-23 (без дублей).csv']

In [67]:
files_cian

['циан 25-04-23 (без дублей).csv',
 'циан 26-04-23 (без дублей).csv',
 'циан 27-04-23 (без дублей).csv',
 'циан 28-04-23 (без дублей).csv',
 'циан 29-04-23 (без дублей).csv',
 'циан 30-04-23 (без дублей).csv',
 'циан 01-05-23 (без дублей).csv',
 'циан 02-05-23 (без дублей).csv',
 'циан 03-05-23 (без дублей).csv',
 'циан 04-05-23 (без дублей).csv',
 'циан 05-05-23 (без дублей).csv',
 'циан 06-05-23 (без дублей).csv',
 'циан 07-05-23 (без дублей).csv',
 'циан 08-05-23 (без дублей).csv',
 'циан 09-05-23 (без дублей).csv',
 'циан 10-05-23 (без дублей).csv',
 'циан 11-05-23 (без дублей).csv',
 'циан 12-05-23 (без дублей).csv',
 'циан 13-05-23 (без дублей).csv',
 'циан 14-05-23 (без дублей).csv',
 'циан 15-05-23 (без дублей).csv',
 'циан 16-05-23 (без дублей).csv',
 'циан 17-05-23 (без дублей).csv',
 'циан 18-05-23 (без дублей).csv',
 'циан 19-05-23 (без дублей).csv']

In [68]:
len(files_avito)

24

In [69]:
len(files_cian)

25

In [70]:
def avito_addr_from_row(row, city_df):
    # добавление города для совпадения форматов
    try:
        addr = city_df[city_df.url_avito == row['city']]['name'].values[0]
        addr += '; '
    except:
        addr = ''
    # добавляем район, в авито он может быть None
    try:
        if len(row['Район']) == 0:
            pass
        else:
            addr += row['Район']
            addr += '; '
    except:
        pass
    # добавляем улицу, дом, пр. в авито он может быть None
    try:
        for value in row['Улица'].split(', '):
            addr += value
            addr += '; '
    except:
        pass
    # drop duplicates
    addr = addr[:-2]
    if len(addr.split('; ')) != len(set(addr.split('; '))):
        addr = '; '.join(list(dict.fromkeys(addr.split('; '))))
    return addr

In [71]:
city_df, source_df, exc_code = get_tables_info(sql_engine)

In [72]:
city_df

,id,name,url_avito,population
0,1,Владивосток,vladivostok,0
1,2,Краснодар,krasnodar,0
2,3,Сочи,sochi,0
3,4,Новороссийск,novorossiysk,0
4,5,Анапа,anapa,0
5,6,Армавир,armavir,0
6,7,Челябинск,chelyabinsk,0
7,8,Магнитогорск,magnitogorsk,0
8,9,Миасс,miass,0
9,10,Копейск,kopeysk,0


In [73]:
test_avito['city'] = test_avito['Ссылка'].apply(lambda x: x.split('/')[3])

In [74]:
test_avito['addr'] = test_avito.apply(lambda row: avito_addr_from_row(row, city_df), axis=1)

In [75]:
test_avito

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений,ad_id,city,addr
0,https://www.avito.ru/tomsk/kvartiry/apartament...,"Апартаменты-студия, 29 м², 5/5 эт.",Апартаменты-студия,29 м²,5/5 эт.,1975000,68103,NaN,р-н Ленинский,"микрорайон Черемошники, Кузнецкий пер., 5",12 часов назад,Частное лицо,NaN,NaN,NaN,3169985711,tomsk,Томск; р-н Ленинский; микрорайон Черемошники; ...
1,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 34,9 м², 4/6 эт.",1-к. квартира,"34,9 м²",4/6 эт.,3700000,106017,NaN,р-н Кировский,"ул. Матросова, 6",6 дней назад,Частное лицо,NaN,NaN,NaN,2739762184,tomsk,Томск; р-н Кировский; ул. Матросова; 6
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 36 м², 1/2 эт.",2-к. квартира,36 м²,1/2 эт.,2600000,72222,NaN,р-н Октябрьский,"ул. Мичурина, 99Ас4",22 часа назад,Агентство,ОФИС ПРОДАЖ «ЗАСТРОЙЩИКА»,https://www.avito.ru/user/c245de34b6855769043d...,388,2861453708,tomsk,Томск; р-н Октябрьский; ул. Мичурина; 99Ас4
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 37 м², 3/5 эт.",2-к. квартира,37 м²,3/5 эт.,3350000,90541,NaN,р-н Ленинский,"ул. Говорова, 18",4 часа назад,Частное лицо,NaN,NaN,NaN,2859068275,tomsk,Томск; р-н Ленинский; ул. Говорова; 18
4,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 44,8 м², 3/5 эт.",2-к. квартира,"44,8 м²",3/5 эт.,5400000,120536,NaN,р-н Кировский,"ул. Гоголя, 61",2 часа назад,Частное лицо,NaN,NaN,NaN,2920724735,tomsk,Томск; р-н Кировский; ул. Гоголя; 61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200832,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 7/24 эт.",2-к. квартира,"64,2 м²",7/24 эт.,11877000.0,185000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2919396604,krasnodar,Краснодар; р-н Карасунский; ул. Новороссийская...
200833,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 8/24 эт.",2-к. квартира,"64,2 м²",8/24 эт.,11877000.0,185000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2918873627,krasnodar,Краснодар; р-н Карасунский; ул. Новороссийская...
200834,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 6/24 эт.",2-к. квартира,"64,2 м²",6/24 эт.,11877000.0,185000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2918970728,krasnodar,Краснодар; р-н Карасунский; ул. Новороссийская...
200835,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 64,2 м², 5/24 эт.",2-к. квартира,"64,2 м²",5/24 эт.,11556000.0,180000.0,"ЖК ""Рекорд"" 2 очередь",р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Застройщик,"ЖК ""Рекорд"" 2 очередь",https://www.avito.ru/user/9baab164afea22c5cf7a...,NaN,2919298481,krasnodar,Краснодар; р-н Карасунский; ул. Новороссийская...


In [81]:
len(list(test_avito.addr.unique()))

33958

In [79]:
# avito
diff_list = []
for i in range(len(files_avito)):
    if i - 1 >=0:
        print('compare ' + files_avito[i], files_avito[i-1])
        test_avito_1 = pd.read_csv('saved_shit/' + files_avito[i], delimiter=';')
        test_avito_1['ad_id'] = test_avito_1['Ссылка'].apply(lambda x: x.split('_')[-1])
        test_avito_1['ad_id'] = test_avito_1['ad_id'].astype('int64')
        test_avito_1['city'] = test_avito['Ссылка'].apply(lambda x: x.split('/')[3])
        test_avito_1['addr'] = test_avito_1.apply(lambda row: avito_addr_from_row(row, city_df), axis=1)
        

        test_avito_pre = pd.read_csv('saved_shit/' + files_avito[i - 1], delimiter=';')
        test_avito_pre['ad_id'] = test_avito_pre['Ссылка'].apply(lambda x: x.split('_')[-1])
        test_avito_pre['ad_id'] = test_avito_pre['ad_id'].astype('int64')
        test_avito_pre['city'] = test_avito['Ссылка'].apply(lambda x: x.split('/')[3])
        test_avito_pre['addr'] = test_avito_1.apply(lambda row: avito_addr_from_row(row, city_df), axis=1)

        # рассчет по записям
        
        diff_ad_id = len(set(test_avito_1.ad_id.to_list()) - set(test_avito_pre.ad_id.to_list()))

        print(diff_ad_id, 'записей между 2 файлами; в рублях:                       ', diff_ad_id * 0.15)
        
        diff_all_ad_id = len(set(test_avito_1.ad_id.to_list()) - set(all_ad_id))
        print(diff_all_ad_id, 'записей между новым файлом и всеми остальными; в рублях: ', diff_all_ad_id * 0.15)
        all_ad_id.extend(test_avito_1.ad_id.to_list())
        
        # рассчет по адресам
        
        
        
        print()
    else: # first file
        test_avito_new = pd.read_csv('saved_shit/' + files_avito[i], delimiter=';')
        test_avito_new['ad_id'] = test_avito_new['Ссылка'].apply(lambda x: x.split('_')[-1])
        test_avito_new['ad_id'] = test_avito_new['ad_id'].astype('int64')
        
        all_ad_id = test_avito_new.ad_id.to_list()
        
        print('первый файл - все новые: ', len(set(test_avito_new.ad_id.to_list())))
        print()

первый файл - все новые:  172756

compare 26-04-23 (без дублей).csv 25-04-23 (без дублей).csv
18960 записей между 2 файлами; в рублях:                        2844.0
18960 записей между новым файлом и всеми остальными; в рублях:  2844.0

compare 27-04-23 (без дублей).csv 26-04-23 (без дублей).csv
12043 записей между 2 файлами; в рублях:                        1806.45
6829 записей между новым файлом и всеми остальными; в рублях:  1024.35

compare 28-04-23 (без дублей).csv 27-04-23 (без дублей).csv
25723 записей между 2 файлами; в рублях:                        3858.45
16228 записей между новым файлом и всеми остальными; в рублях:  2434.2

compare 29-04-23 (без дублей).csv 28-04-23 (без дублей).csv
19246 записей между 2 файлами; в рублях:                        2886.9
6237 записей между новым файлом и всеми остальными; в рублях:  935.55

compare 30-04-23 (без дублей).csv 29-04-23 (без дублей).csv
13862 записей между 2 файлами; в рублях:                        2079.2999999999997
2453 запис

In [87]:
# cian
diff_list = []
for i in range(len(files_cian)):
    if i - 1 >=0:
        print('compare ' + files_cian[i], files_cian[i-1])
        test_cian_1 = pd.read_csv('saved_shit/' + files_cian[i], delimiter=',')
        test_cian_1['ad_id'] = test_cian_1['Ссылка'].apply(lambda x: x.split('/')[-2])
        test_cian_1['ad_id'] = test_cian_1['ad_id'].astype('int64')

        test_cian_pre = pd.read_csv('saved_shit/' + files_cian[i - 1], delimiter=',')
        test_cian_pre['ad_id'] = test_cian_pre['Ссылка'].apply(lambda x: x.split('/')[-2])
        test_cian_pre['ad_id'] = test_cian_pre['ad_id'].astype('int64')

        diff_ad_id = len(set(test_cian_1.ad_id.to_list()) - set(test_cian_pre.ad_id.to_list()))

        print(diff_ad_id, 'записей между 2 файлами; в рублях:                       ', diff_ad_id * 0.15)
        
        diff_all_ad_id = len(set(test_cian_1.ad_id.to_list()) - set(all_ad_id))
        print(diff_all_ad_id, 'записей между новым файлом и всеми остальными; в рублях: ', diff_all_ad_id * 0.15)
        all_ad_id.extend(test_cian_1.ad_id.to_list())
        print()
    else: # first file
        test_cian_new = pd.read_csv('saved_shit/' + files_cian[i], delimiter=',')
        test_cian_new['ad_id'] = test_cian_new['Ссылка'].apply(lambda x: x.split('/')[-2])
        test_cian_new['ad_id'] = test_cian_new['ad_id'].astype('int64')
        
        all_ad_id = test_cian_new.ad_id.to_list()
        
        print('первый файл - все новые: ', len(set(test_avito_new.ad_id.to_list())))
        print()

первый файл - все новые:  172756

compare циан 26-04-23 (без дублей).csv циан 25-04-23 (без дублей).csv
5751 записей между 2 файлами; в рублях:                        862.65
5751 записей между новым файлом и всеми остальными; в рублях:  862.65

compare циан 27-04-23 (без дублей).csv циан 26-04-23 (без дублей).csv
32326 записей между 2 файлами; в рублях:                        4848.9
9316 записей между новым файлом и всеми остальными; в рублях:  1397.3999999999999

compare циан 28-04-23 (без дублей).csv циан 27-04-23 (без дублей).csv
9441 записей между 2 файлами; в рублях:                        1416.1499999999999
4697 записей между новым файлом и всеми остальными; в рублях:  704.55

compare циан 29-04-23 (без дублей).csv циан 28-04-23 (без дублей).csv
10451 записей между 2 файлами; в рублях:                        1567.6499999999999
2538 записей между новым файлом и всеми остальными; в рублях:  380.7

compare циан 30-04-23 (без дублей).csv циан 29-04-23 (без дублей).csv
8297 записей ме

In [80]:
files_avito[0]

'25-04-23 (без дублей).csv'

In [81]:
files_avito[-1]

'08-05-23 (без дублей).csv'

In [82]:
test_avito_1 = pd.read_csv('saved_shit/' + files_avito[0], delimiter=';')
test_avito_1['ad_id'] = test_avito_1['Ссылка'].apply(lambda x: x.split('_')[-1])
test_avito_1['ad_id'] = test_avito_1['ad_id'].astype('int64')

test_avito_last = pd.read_csv('saved_shit/' + files_avito[-1], delimiter=';')
test_avito_last['ad_id'] = test_avito_last['Ссылка'].apply(lambda x: x.split('_')[-1])
test_avito_last['ad_id'] = test_avito_last['ad_id'].astype('int64')

In [83]:
set(test_avito_1.ad_id.to_list()) - set(test_avito_last.ad_id.to_list())

{2950299649,
 2602303494,
 2433613835,
 2868903949,
 2782265362,
 2892759059,
 2919104530,
 2858549269,
 2752774165,
 2393899032,
 2466775066,
 2791440416,
 2670198822,
 2824601644,
 2985951277,
 2912157743,
 2851733552,
 2668494899,
 2951086135,
 2970615868,
 2892759104,
 2613837890,
 2919497794,
 2700476485,
 2872049736,
 2906783819,
 2588541006,
 2950037584,
 2618949713,
 2531524689,
 2996699220,
 2748448858,
 2937978979,
 2748055653,
 2921070697,
 2901934188,
 2427715692,
 2700345453,
 2570322030,
 2872311922,
 2759196788,
 2591686778,
 2742681722,
 2409365633,
 2606104706,
 2920939651,
 2730098826,
 2891972747,
 2944663691,
 2711486608,
 2548301970,
 2911502486,
 2810445974,
 2624192664,
 2855272602,
 2887516315,
 2780561565,
 2864840862,
 2694578333,
 2987786398,
 2625765537,
 2654470309,
 2720792742,
 2670198953,
 2740715689,
 2192310442,
 2608857263,
 2538602672,
 2869952689,
 2759327921,
 3027894452,
 2875982005,
 2918842550,
 2776760503,
 2805072064,
 2766143681,
 2709389505,